<h1 style="background-color:DodgerBlue;color:White;padding:20px;text-align: center;line-height:1.5;"> <b>Import nhiều files excel cùng lúc hoặc <br> nhiều sheets của 1 file excel vào Stata</b> </h1>

<i>Đào Huy Cừ</i><br>
<i>Ngày tháng: 09 tháng 6 năm 2020</i>

<h2><b> Import nhiều file excel trong cùng 1 folder </b></h2>

<p> Viết chương trình loops qua và import tất cả các files </p>

In [28]:
set linesize 165

In [1]:
cap program drop myimport
program define myimport 
version 15
args ddang noifiles
global filenames : dir"."files"*.`ddang'"

foreach filename of global filenames {
	local filename1 = subinstr("`filename'"," ","",.)        
	local filename2 = subinstr("`filename1'",".`ddang'","",.) 
    if ("`ddang'" == "xlsx") | ("`ddang'"== "xls") {
        qui import excel using `"`filename'"', first all clear
        qui save `"`filename2'.dta"', replace
        clear
    }
    if ("`ddang'" == "csv") {
        qui import delimited using `"`filename'"', clear
        qui save `"`filename2'.dta"', replace
        clear
    }
}
// Nối các dta files lại với nhau
if "`noifiles'"!="" {
local files:dir"."files"*.dta"
local listfiles = ""
foreach f of local files {
    local listfiles = "`listfiles'" + " `f'"
}
local w1      = word("`listfiles'",1)
local wconlai = subinword("`listfiles'","`w1'","",1) //Stata 15.1 SE
qui use `w1', clear
qui append using `wconlai', gen(varmark)
di as text "all dta files are appended, " _newline "variable varmark are generated from option gen() of append command"
}
end

In [2]:
// Ví dụ về 1 folder có chứa 48 files số liệu có cấu trúc giống nhau ở các biến số thu thập và format trong
mydir
qui cd "$hrdata"
dir *.csv




  18.1k   2/12/19 10:37  201501.csv        
  18.2k   2/12/19 10:37  201502.csv        
  18.2k   2/12/19 10:38  201503.csv        
  18.5k   2/12/19 10:38  201504.csv        
  19.1k   2/12/19 10:39  201505.csv        
  19.9k   2/12/19 10:39  201506.csv        
  20.4k   2/12/19 10:39  201507.csv        
  21.2k   2/12/19 10:40  201508.csv        
  21.7k   2/12/19 10:40  201509.csv        
  22.8k   2/12/19 10:40  201510.csv        
  23.6k   2/12/19 10:40  201511.csv        
  24.0k   2/12/19 10:41  201512.csv        
  24.9k   2/12/19 10:41  201601.csv        
  25.5k   2/12/19 10:41  201602.csv        
  26.9k   2/12/19 10:41  201603.csv        
  27.6k   2/12/19 10:42  201604.csv        
  28.1k   2/12/19 10:42  201605.csv        
  28.7k   2/12/19 10:42  201606.csv        
  29.3k   2/12/19 10:42  201607.csv        
  30.7k   2/12/19 10:43  201608.csv        
  32.4k   2/12/19 10:43  201609.csv        
  34.2k   2/12/19 10:43  201610.csv        
  36.8k   2/12/19 10:43  2016

In [3]:
// Xóa những file dta trong folder
clear
local dtafiles: dir"."files"*.dta"
foreach v of local dtafiles {
    erase `v'
}

In [4]:
// Import tat ca cac files trong folder va luu lai file so lieu cung ten
myimport csv noifiles

all dta files are appended, 
variable varmark are generated from option gen() of append command


In [5]:
tab varmark


    varmark |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |        228        1.03        1.03
          1 |        229        1.03        2.07
          2 |        229        1.03        3.10
          3 |        233        1.05        4.15
          4 |        242        1.09        5.25
          5 |        251        1.13        6.38
          6 |        258        1.17        7.55
          7 |        269        1.22        8.76
          8 |        275        1.24       10.00
          9 |        289        1.31       11.31
         10 |        298        1.35       12.66
         11 |        304        1.37       14.03
         12 |        316        1.43       15.46
         13 |        324        1.46       16.92
         14 |        342        1.55       18.47
         15 |        350        1.58       20.05
         16 |        357        1.61       21.66
         17 |        364        1.64       23.31
         18 |      

<h2>Một cách khác là sử dụng <i>xls2dta</i> command</h2>

In [9]:
// Nếu chưa cài đặt câu lệnh này thì cần tải nó xuống.
cap which xls2dta
if _rc ssc install xls2dta

In [33]:
dir *.xls

  64.5k   6/17/20 11:00  201501.xls        
  65.0k   6/17/20 11:00  201502.xls        
  65.0k   6/17/20 11:00  201503.xls        
  66.0k   6/17/20 11:00  201504.xls        
  68.5k   6/17/20 11:00  201505.xls        


In [31]:
qui local wd: pwd
qui xls2dta, save("`wd'", replace): import excel "`wd'"
qui xls2dta, save("`wd'\completed.dta", replace): append

In [32]:
use completed.dta, clear
count 



  1,166


<h2 style="color: red;line-height:1.5;">Ngoài ra xls2dta còn có thể import và nối số liệu ở nhiều sheet trong cùng 1 files<br>Tuy nhiên hạn chế là không import được file định dạng csv</h2>